In [ ]:
import pandas as pd

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# Data loading
test_data = pd.read_csv("/content/drive/MyDrive/Automated Debate Scoring/test_set.tsv",sep='\t', encoding='ISO-8859-1', quoting=3, error_bad_lines=False)
valid_data = pd.read_csv("/content/drive/MyDrive/Automated Debate Scoring/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
train_data = pd.read_csv("/content/drive/MyDrive/Automated Debate Scoring/training_set_rel3.tsv", sep='\t', encoding = 'ISO-8859-1', quoting=3, error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


FileNotFoundError: ignored

In [ ]:
test_data.dropna(axis=1,inplace=True)
valid_data.dropna(axis=1,inplace=True)

In [ ]:
# Setting data
X_data = train_data['essay']
Y_data = train_data['domain1_score']

X_data.drop_duplicates()
Y_data.drop_duplicates()

0         8
1         9
2         7
3        10
13        6
15       12
18        4
21        3
23       11
40        2
49        5
1784      1
3590      0
10686    15
10687    13
10689    17
10691    23
10692    16
10693    18
10697    19
10699    14
10704    21
10707    24
10722    20
10752    22
12255    34
12256    46
12257    40
12258    30
12259    26
12263    41
12265    31
12266    44
12268    36
12269    43
12271    45
12274    35
12276    42
12278    33
12279    38
12287    47
12290    32
12297    50
12298    39
12307    37
12337    55
12342    60
12344    28
12345    49
12347    29
12352    27
12507    25
12526    48
Name: domain1_score, dtype: int64

In [ ]:
def essay_to_wordlist(essay_v, remove_stopwords):
    #Remove the tagged labels and word tokenize the sentence.
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make ar from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 38)

In [ ]:
from gensim.models import Word2Vec
import numpy as np
import nltk
import re
nltk.download('punkt')

sentences = []
for essay in X_train:
    # Obtaining all sentences from the training essays.
    sentences += essay_to_sentences(essay, remove_stopwords = True)

# Initializing variables for word2vec model.
num_features = 300
min_word_count = 40
num_workers = 8
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

clean_train_essays = []   
# Generate training and testing data word vectors.
for essay_v in X_train:
    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
clean_test_essays = []
for essay_v in X_test:
    clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)
trainDataVecs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NameError: ignored

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score

cv = KFold(n_splits = 5, shuffle = True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X_data):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X_data.iloc[testcv], X_data.iloc[traincv], Y_data.iloc[testcv], Y_data.iloc[traincv]
    
    train_essays = X_train
    test_essays = X_test
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 300
    min_word_count = 40
    num_workers = 8
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )

    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep) 
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=2)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 5 models.
    if count == 5:
         lstm_model.save_weights('final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1

NameError: ignored

In [ ]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.7426
